# Exercise: Computational Linguistics over Reddit Data

For this project we are going to ingest Reddit posts, process the data and perform computational linguistics over the Reddit posts.

This project will build off of some work you have previously done. However, beyond that exercise of processing and cataloging the feeds, in this instance you will access the referenced Reddit post and perform computational linguistics over the post itself.

![DataScraper_To_NLP.png MISSING](../images/DataScraper_To_NLP.png)

---

### From the site:

reddit: https://www.reddit.com/  
Reddit gives you the best of the Internet in one place. Get a constantly updating feed of breaking news, fun stories, pics, memes, and videos just for you.


### From Wikipedia:
Reddit is an American social news aggregation, web content rating, and discussion website. 
Registered members submit content to the site such as links, text posts, and images, 
which are then voted up or down by other members. 
Posts are organized by subject into user-created boards called "subreddits", 
which cover a variety of topics including news, science, movies, video games, music, books, fitness, food, and image-sharing. 
Submissions with more up-votes appear towards the top of their subreddit and, if they receive enough votes, ultimately on the site's front page. 



#### Sample Posting:

The below link is an example post from someone that was tinkering with sentiment analysis; specifically they looked at the text of [Moby Dick](https://en.wikipedia.org/wiki/Moby-Dick).

**Spoiler:** The conclusion was that the book is rather negative in sentiment.
It is after all, about vengeance!

https://www.reddit.com/r/LanguageTechnology/comments/9whk23/a_simple_nlp_pipeline_to_calculate_running/



### From: https://www.redditinc.com/
![REDDIT_About.png MISSING](../images/REDDIT_About_latest.png)

---

## Data Acquisition


### Example Code:

In this exercise, we will be using Reddit API for fetching the latest messages. We can also fetch recent posts from Reddit using web feeds (check [here](./rss-feeds.ipynb)), but it seems our IP got banned for excessive requests to Reddit over the last few days. So we will be using Reddit API for which you are required to create your Reddit account and an app. 

Follow [this article](https://gilberttanner.com/blog/scraping-redditdata) to create your credentials. 

### Using Reddit API

For fetching Reddit data using API, we will be using a Python wrapper to Reddit API: [PRAW: The Python Reddit API Wrapper](https://github.com/praw-dev/praw)

Documentation: https://praw.readthedocs.io/en/latest/index.html

In [1]:
import pandas as pd
import praw
import datetime

reddit = praw.Reddit(client_id='3yY7G906Udk--SEIM6qbug', 
                     client_secret='0t8wt0AZBFPBdIXTIlEuFESK4XVApA',
                     user_agent='James_Webscrapper')

Version 7.4.0 of praw is outdated. Version 7.7.1 was released Tuesday July 11, 2023.


In [2]:
hot_posts = reddit.subreddit('datascience').hot(limit=10)
all_posts = list(hot_posts)

In [3]:
for post in all_posts:
    print(f"id : {post.id}")
    print(f"title : {post.title}")
    print(f"url : {post.url}")
    print(f"author : {str(post.author)} {type(str(post.author))}")
    print(f"score : {post.score} {type(post.score)} ")
    print(f"subreddit : {post.subreddit} {type(post.subreddit)} ")
    print(f"num_comments : {post.num_comments}")
    print(f"body : {post.selftext}")
    print(f"created : {post.created}")
    print(f"link_flair_text : {post.link_flair_text}")
    break

id : 1fxz22n
title : Weekly Entering & Transitioning - Thread 07 Oct, 2024 - 14 Oct, 2024
url : https://www.reddit.com/r/datascience/comments/1fxz22n/weekly_entering_transitioning_thread_07_oct_2024/
author : AutoModerator <class 'str'>
score : 1 <class 'int'> 
subreddit : datascience <class 'praw.models.reddit.subreddit.Subreddit'> 
num_comments : 56
body :  

Welcome to this week's entering & transitioning thread! This thread is for any questions about getting started, studying, or transitioning into the data science field. Topics include:

* Learning resources (e.g. books, tutorials, videos)
* Traditional education (e.g. schools, degrees, electives)
* Alternative education (e.g. online courses, bootcamps)
* Job search questions (e.g. resumes, applying, career prospects)
* Elementary questions (e.g. where to start, what next)

While you wait for answers from the community, check out the [FAQ](https://www.reddit.com/r/datascience/wiki/frequently-asked-questions) and Resources pages on

### Sub-Reddits

As described above, sub-reddits are communities organized around particular topics.

Some example sub-reddits:
 * https://www.reddit.com/r/datascience/
 * https://www.reddit.com/r/MachineLearning/
 * https://www.reddit.com/r/LanguageTechnology/
 * https://www.reddit.com/r/NLP/
 * https://www.reddit.com/r/Python/


# Exercise Tasks

## Part I: Data Acquisition and Loading 
1. Choose a subreddit of your choice. Preferably something of interest to you. 
1. Conceptualize a database design that can collect the data.
    * Make sure your items (posts) are unique and not duplicated!
    * Make sure you capture at least title, author, subreddit, tags, title link, and timestamp
    * Along with the metadata, capture all the text into one or more data field(s) suitable for information retrieval
    * Write triggers for auto updates of IR related fields
    * Add index (either GIN or GiST) for the IR related fields
    * Additionally, design a field to hold:
        * Sentiment
1. Implement the database in your PostgreSQL schema
1. Implement cells of Python Code that 
    * collect the latest posts from a subreddit of your choice (**should be text-dominant not image/video**) and collect at least 500 posts (if possible), 
    * processes the messages to extract metadata, 
    * process the text for IR, and 
    * perform computational linguistics (i.e, extract sentiment scores), 
    * then insert the data into your database.
1. After you have loaded data from a subreddit, choose a few more subreddits and load those!

## Part II: Analytics 

1. Write some test queries following the text vectors from Module 7.
1. Produce **interesting visualizations** of the linguistic data.
    * Try to look for trends (within a subreddit) and and variations of topics across subreddits
    * Some comparative plots across feeds
1. Write a summary of your findings!

 
 

# Part I: Data Acquisition and Loading

## Task 1: Design your database

Conceptualize a database design that can collect the data.
* Make sure your items (posts) are unique and not duplicated!
* Make sure you capture at least title, link, author, subreddit, tag/flair, and timestamp
* Capture all the body text into fields suitable for information retrieval
* Write triggers for auto updates of IR related fields
* Add index (either GIN or GiST) for the IR related fields
* Additionally, design a field to hold:
    - Sentiment


In [4]:
posts = []
ds_subreddit = reddit.subreddit('datascience')

for post in ds_subreddit.hot(limit=10):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments,
                  post.selftext, datetime.datetime.fromtimestamp(post.created)])

posts_df = pd.DataFrame(posts, columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments',
                                        'body', 'created'])

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

posts_df

,title,score,id,subreddit,url,num_comments,body,created
0,Weekly Entering & Transitioning - Thread 07 Oc...,1,1fxz22n,datascience,https://www.reddit.com/r/datascience/comments/...,56,\n\nWelcome to this week's entering & transit...,2024-10-06 23:01:31
1,I created a 6-week SQL for data science roadma...,372,1fzswk1,datascience,https://www.reddit.com/r/datascience/comments/...,34,I created this roadmap to guide you through ma...,2024-10-09 09:36:10
2,SQL queries that group by number,10,1g01l33,datascience,https://www.reddit.com/r/datascience/comments/...,31,I wanted to know if generally people use group...,2024-10-09 15:41:19
3,"Need help on analysis of AI performance, compu...",6,1fzw393,datascience,https://www.reddit.com/gallery/1fzw393,4,,2024-10-09 11:50:22
4,I linked AI Performance Data with Compute Size...,0,1g07nvw,datascience,https://www.reddit.com/gallery/1g07nvw,5,,2024-10-09 20:27:28
5,Which position should I join? (Palantir Develo...,53,1fzh90w,datascience,https://www.reddit.com/r/datascience/comments/...,58,I have recently received two offers from two d...,2024-10-08 21:26:32
6,Does business dictate what models or methodolo...,6,1fznew0,datascience,https://www.reddit.com/r/datascience/comments/...,6,"Hey guys,\n\nI am working on a forecasting pro...",2024-10-09 04:30:56
7,does anyone use Posit Connect?,16,1fzi4an,datascience,https://www.reddit.com/r/datascience/comments/...,12,I'm curious what companies out there are using...,2024-10-08 22:13:56
8,A guide to passing the A/B test interview ques...,920,1fyrawz,datascience,https://www.reddit.com/r/datascience/comments/...,77,"Hey all,\n\nI'm a Sr. Analytics Data Scientist...",2024-10-07 23:08:05
9,The Nobel Prize in Physics 2024 was awarded to...,59,1fz6o4i,datascience,/r/math/comments/1fyzz6t/the_nobel_prize_in_ph...,13,,2024-10-08 13:19:00



---

## Task 2: Implement the database in your PostgreSQL schema

You can choose any of the three ways to implement your database. 

* sql magic 
* sql terminal 
* psycopg2 or sqlalchemy


In [5]:
import getpass
import psycopg2
mypasswd = getpass.getpass()

········


In [6]:
mysso = 'jsmm8'
dbname = 'dsa_student'
schema = 'jsmm8'

In [7]:
import psycopg2
import numpy as np
from psycopg2.extensions import adapt, register_adapter, AsIs

connection = psycopg2.connect(database = dbname, 
                              user = mysso, 
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

In [8]:
CREATE_TABLES = f"""
-------------------------
-- Basic Table 
-------------------------
DROP TABLE IF EXISTS {schema}.data_science_posts;

CREATE TABLE {schema}.data_science_posts(
    id SERIAL PRIMARY KEY,
    post_id VARCHAR(255) UNIQUE NOT NULL,
    title TEXT NOT NULL,
    author VARCHAR(255) NOT NULL,
    subreddit VARCHAR(255) NOT NULL,
    tags TEXT[],
    title_link TEXT NOT NULL,
    timestamp TIMESTAMP NOT NULL,
    processed_body TEXT,
    sentiment VARCHAR(50),
    ir_field TSVECTOR,
    created_at TIMESTAMP DEFAULT NOW()
);

"""

In [9]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_TABLES)

In [10]:
create_trigger_function_query = '''
CREATE OR REPLACE FUNCTION update_ir_field()
RETURNS TRIGGER AS $$
BEGIN
    NEW.ir_field := to_tsvector('english', COALESCE(NEW.title, '') || ' ' || COALESCE(NEW.post_body, ''));
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;
'''

In [11]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_TABLES)

In [12]:
create_trigger_query = '''
CREATE TRIGGER ir_field_update
BEFORE INSERT OR UPDATE ON data_science_posts
FOR EACH ROW
EXECUTE FUNCTION update_ir_field();
'''

In [13]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_TABLES)

In [14]:
create_index_query = '''
CREATE INDEX IF NOT EXISTS ir_field_idx ON data_science_posts USING GIN(ir_field);
'''

In [15]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_TABLES)

In [16]:
SQL_QUERY = f"""
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema = '{schema}' AND table_name = 'data_science_posts';
"""

In [17]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_TABLES)

In [18]:
df = pd.read_sql_query(SQL_QUERY, connection)
df

,table_schema,table_name,column_name,data_type
0,jsmm8,data_science_posts,id,integer
1,jsmm8,data_science_posts,post_id,character varying
2,jsmm8,data_science_posts,title,text
3,jsmm8,data_science_posts,author,character varying
4,jsmm8,data_science_posts,subreddit,character varying
5,jsmm8,data_science_posts,tags,ARRAY
6,jsmm8,data_science_posts,title_link,text
7,jsmm8,data_science_posts,timestamp,timestamp without time zone
8,jsmm8,data_science_posts,processed_body,text
9,jsmm8,data_science_posts,sentiment,character varying


## Task 3: Implement cells of Python Code that

* collect the latest posts from a subreddit of your choice (should be text-dominant not image/video) and collect at least 500 posts (if possible),
* processes the messages to extract id, title, link, author, subreddit, tag/flair, timestamp, etc. 
* process the text for IR, and
* perform computational linguistics (e.g., get sentiment scores)
* then insert the data into your database.


Notes: 
* Each call to Reddit API returns 100 entries max. If we set a limit of more than 100, PRAW will handle multiple API calls internally and lazily fetches data. Check obfuscation and API limitation in https://praw.readthedocs.io/en/v3.6.2/pages/getting_started.html. 
* Develop and test your code with less than 100 messages from a subreddit. Then increase the limit and add few more subreddits. 
* While loading the table, test with one row 


In [19]:
## Your code in this cell
## ------------------------
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import datetime

connection = psycopg2.connect(database=dbname, 
                              user=mysso, 
                              host='pgsql.dsa.lan',
                              password=mypasswd)
cur = connection.cursor()

reddit = praw.Reddit(client_id='3yY7G906Udk--SEIM6qbug', 
                     client_secret='0t8wt0AZBFPBdIXTIlEuFESK4XVApA',
                     user_agent='James_Webscrapper')

subreddit_name = 'datascience'
ds_subreddit = reddit.subreddit(subreddit_name)
analyzer = SentimentIntensityAnalyzer()

def classify_sentiment(compound_score):
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

posts = []
for post in ds_subreddit.hot(limit=500):
    body_text = post.selftext
    sentiment_scores = analyzer.polarity_scores(body_text)
    sentiment_label = classify_sentiment(sentiment_scores['compound'])
    
    posts.append({
        'post_id': post.id,
        'title': post.title,
        'author': str(post.author) if post.author else 'Unknown',
        'subreddit': post.subreddit.display_name,
        'url': post.url,
        'tags': post.link_flair_text,
        'created': datetime.datetime.fromtimestamp(post.created),
        'processed_body': body_text,
        'sentiment': sentiment_label,
        'compound': sentiment_scores['compound']
    })

print(f"Fetched {len(posts)} posts.")

     |████████████████████████████████| 133kB 3.8MB/s eta 0:00:01
Fetched 151 posts.


In [20]:
for post in posts:
    sentiment_scores = analyzer.polarity_scores(post['processed_body'])
    print(f"Negative: {sentiment_scores['neg']}")
    print(f"Neutral: {sentiment_scores['neu']}")
    print(f"Positive: {sentiment_scores['pos']}")
    print(f"Compound: {sentiment_scores['compound']}")
    print(f"Body: {post['processed_body']}\n")


Negative: 0.0
Neutral: 0.944
Positive: 0.056
Compound: 0.6696
Body:  

Welcome to this week's entering & transitioning thread! This thread is for any questions about getting started, studying, or transitioning into the data science field. Topics include:

* Learning resources (e.g. books, tutorials, videos)
* Traditional education (e.g. schools, degrees, electives)
* Alternative education (e.g. online courses, bootcamps)
* Job search questions (e.g. resumes, applying, career prospects)
* Elementary questions (e.g. where to start, what next)

While you wait for answers from the community, check out the [FAQ](https://www.reddit.com/r/datascience/wiki/frequently-asked-questions) and Resources pages on our wiki. You can also search for answers in [past weekly threads](https://www.reddit.com/r/datascience/search?q=weekly%20thread&restrict_sr=1&sort=new).

Negative: 0.0
Neutral: 0.814
Positive: 0.186
Compound: 0.9694
Body: I created this roadmap to guide you through mastering SQL in about 6 

Negative: 0.085
Neutral: 0.807
Positive: 0.109
Compound: 0.8999
Body: In my experience, when people consider applying LLMs to a project they often fall into two camps:

1. they turn the project into a chat bot
2. they use an LLM for some key feature in a larger application, resulting in an error prone mess

there's tremendous power in using LLMs to power specific features within larger applications, but LLMs inconsistency in output structure makes it difficult to use their output within a programmatic system. You might ask an llm to output JSON data, for instance, and the LLM decides it's appropriate to wrap the data in a `\`\`\`json \`\`\`` markdown format. you might ask an LLM to output a list of values, and it responds with something like this:

    here's your list
    [1,2,3,4]

There's an infinite number of ways LLM output can go wrong, which is why output parsing is a thing.

I've had the best luck, personally, with LangChain in this regard. [LangChain's pydantic parser](https:/

Negative: 0.056
Neutral: 0.793
Positive: 0.151
Compound: 0.9921
Body: Hey everyone,

I'm looking for some project suggestions, but I want to avoid the typical ones like credit card fraud detection or Titanic datasets. I feel like those are super common on every DS resume, and I want to stand out a bit more.

I am a B. Applied CS student (Stats Minor) and I'm especially interested in Data Engineering (DE), Data Science (DS), or Machine Learning (ML) projects, As I am targeting DS/DA roles for my co-op. Unfortunately, I haven’t found many interesting projects so far. They mention all the same projects, like customer churn, stock prediction etc.  
  
I’d love to explore projects that showcase tools and technologies beyond the usual suspects I’ve already worked with (numpy, pandas, pytorch, SQL, python, tensorflow, Foleum, Seaborn, Sci-kit learn, matplotlib).

I’m particularly interested in working with tools like PySpark, Apache Cassandra, Snowflake, Databricks, and anything else along th

Neutral: 0.934
Positive: 0.036
Compound: -0.2477
Body: Right now, I have 3 job offers on the table. One from the Patent+Trademark Office, one from the FCC, and one from a private gov't consulting firm. I don't think I'll take the one from the PTO, but the FCC/Private choice has me hung up. 


The FCC job is my current shortest path to my goal: federal work as high as I can handle. I'd prefer the GAO to anything else, but I'll take what I can get. The work at the FCC isn't particularly "data science"-y either, but it starts my climb up the GS scale so that's a big 'plus.


The private job salary beats the FCC one by a lot (~$30k), and it's for sure nuts-and-bolts data science doing text modelling (something that's also a goal of mine: staying technically on the forefront). But it doesn't really get me much closer to my main goal of federal work.


My favorite 'job lead' I have right now is currently in scheduling hell and probably won't come out of that before I actually need to start h

Negative: 0.035
Neutral: 0.9
Positive: 0.064
Compound: 0.9879
Body: Hey folks,

I'm a Sr. Analytics Data Scientist at a large tech firm (not FAANG) and I conduct about \~3 interviews per week. I wanted to share my transition to data science in case it helps other folks, as well as share my advice for how to nail the product analytics interviews. I also want to raise awareness that Product Analytics is a very viable and lucrative data science path. I'm not going to get into the distinction between analytics and data science/machine learning here. Just know that I don't do any predictive modeling, and instead do primarily AB testing, causal inference, and dashboarding/reporting. I do want to make one thing clear: This advice is primarily applicable to analytics roles in tech. It is probably not applicable for ML or Applied Scientist roles, or for fields other than tech. Analytics roles can be very lucrative, and the barrier to entry is lower than that for Machine Learning roles. The bar 

In [21]:
insert_query = f"""
INSERT INTO {schema}.data_science_posts (post_id, title, author, subreddit, tags, title_link, timestamp, processed_body, sentiment)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (post_id) DO NOTHING;
"""

for post in posts:
    cur.execute(insert_query, (
        post['post_id'], post['title'], post['author'], post['subreddit'], 
        [post['tags']] if post['tags'] else None, post['url'], post['created'], 
        post['processed_body'], post['sentiment']
    ))

print("Data inserted successfully.")

Data inserted successfully.


In [22]:
def print_top_reviews(df, sentiment_label):
    if sentiment_label == 'positive':
        top_reviews = df[df['sentiment'] == 'positive'].sort_values(by='compound', ascending=False).head(3)
    elif sentiment_label == 'negative':
        top_reviews = df[df['sentiment'] == 'negative'].sort_values(by='compound').head(3)
    else:
        neutral_df = df[df['sentiment'] == 'neutral']
        top_reviews = neutral_df.iloc[(neutral_df['compound'] - 0).abs().argsort()[:3]]
    
    print(f"\nTop 3 {sentiment_label.capitalize()} Reviews:")
    for idx, row in top_reviews.iterrows():
        print(f"\nReview {idx+1}:")
        print(f"Compound Score: {row['compound']}")
        print(row['title'])

In [23]:
insert_query = """
INSERT INTO data_science_posts (post_id, title, author, subreddit, title_link, timestamp, processed_body, sentiment)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (post_id) DO NOTHING;
"""
with connection, connection.cursor() as cursor:
    for post in posts:
        cursor.execute(insert_query, (
            post['post_id'],
            post['title'],
            post['author'],
            post['subreddit'],
            post['url'],
            post['created'],
            post['processed_body'],
            post['sentiment']
        ))

print("All posts inserted successfully.")

All posts inserted successfully.


In [24]:
select_query = """
SELECT post_id, title, author, subreddit, timestamp, processed_body, sentiment
FROM data_science_posts
LIMIT 10;
"""

with connection, connection.cursor() as cursor:
    cursor.execute(select_query)
    rows = cursor.fetchall()

for row in rows:
    print(f"Post ID: {row[0]}")
    print(f"Title: {row[1]}")
    print(f"Author: {row[2]}")
    print(f"Subreddit: {row[3]}")
    print(f"Timestamp: {row[4]}")
    print(f"body: {row[5]}")
    print(f"Sentiment: {row[6]}")
    print("-" * 50)

Post ID: 1fxz22n
Title: Weekly Entering & Transitioning - Thread 07 Oct, 2024 - 14 Oct, 2024
Author: AutoModerator
Subreddit: datascience
Timestamp: 2024-10-06 23:01:31
body:  

Welcome to this week's entering & transitioning thread! This thread is for any questions about getting started, studying, or transitioning into the data science field. Topics include:

* Learning resources (e.g. books, tutorials, videos)
* Traditional education (e.g. schools, degrees, electives)
* Alternative education (e.g. online courses, bootcamps)
* Job search questions (e.g. resumes, applying, career prospects)
* Elementary questions (e.g. where to start, what next)

While you wait for answers from the community, check out the [FAQ](https://www.reddit.com/r/datascience/wiki/frequently-asked-questions) and Resources pages on our wiki. You can also search for answers in [past weekly threads](https://www.reddit.com/r/datascience/search?q=weekly%20thread&restrict_sr=1&sort=new).
Sentiment: positive
-----------

In [25]:
add_tsvector_columns_query = f"""
ALTER TABLE {schema}.data_science_posts
ADD COLUMN processed_body_tsv_gin TSVECTOR,
ADD COLUMN processed_body_tsv_gist TSVECTOR;
"""

with connection, connection.cursor() as cursor:
    cursor.execute(add_tsvector_columns_query)

print("Added 'processed_body_tsv_gin' and 'processed_body_tsv_gist' columns to the data_science_posts table.")

Added 'processed_body_tsv_gin' and 'processed_body_tsv_gist' columns to the data_science_posts table.


In [26]:
CREATE_TRIGGER=f"""
--TRIGGER
CREATE TRIGGER tsv_gin_update 
    BEFORE INSERT OR UPDATE
    ON {schema}.data_science_posts 
    FOR EACH ROW 
    EXECUTE PROCEDURE 
    tsvector_update_trigger(processed_body_tsv_gin, 'pg_catalog.english', processed_body);

CREATE TRIGGER tsv_gist_update 
    BEFORE INSERT OR UPDATE
    ON {schema}.data_science_posts 
    FOR EACH ROW 
    EXECUTE PROCEDURE 
    tsvector_update_trigger(processed_body_tsv_gist, 'pg_catalog.english', processed_body);
"""

In [27]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_TRIGGER)

In [28]:
CREATE_INDEX=f"""
-------------------------
-- Create Indexes
-------------------------

-- GIN Index on processed_body (for trigram search)
-- CREATE EXTENSION pg_trgm;  -- Ensure this is done by DB Admin
CREATE INDEX data_science_posts_processed_body
ON {schema}.data_science_posts USING GIN(processed_body gin_trgm_ops);

-- GIN Index on processed_body_tsv_gin for full-text search
CREATE INDEX data_science_posts_processed_body_tsv_gin
ON {schema}.data_science_posts USING GIN(processed_body_tsv_gin);

-- GIST Index on processed_body_tsv_gist for full-text search
CREATE INDEX data_science_posts_processed_body_tsv_gist
ON {schema}.data_science_posts USING GIST(processed_body_tsv_gist);
"""

In [29]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_INDEX)

In [30]:
create_extension_query = "CREATE EXTENSION IF NOT EXISTS pg_trgm;"

with connection, connection.cursor() as cursor:
    cursor.execute(create_extension_query)

print("pg_trgm extension created successfully.")

pg_trgm extension created successfully.


In [31]:
select_query = f"""
SELECT id, title, author, subreddit, sentiment
FROM {schema}.data_science_posts
LIMIT 10;
"""

with connection, connection.cursor() as cursor:
    cursor.execute(select_query)
    rows = cursor.fetchall()

for row in rows:
    print(f"ID: {row[0]}")
    print(f"Title: {row[1]}")
    print(f"Author: {row[2]}")
    print(f"Subreddit: {row[3]}")
    print(f"Sentiment: {row[4]}")
    print("-" * 50)

ID: 1
Title: Weekly Entering & Transitioning - Thread 07 Oct, 2024 - 14 Oct, 2024
Author: AutoModerator
Subreddit: datascience
Sentiment: positive
--------------------------------------------------
ID: 2
Title: I created a 6-week SQL for data science roadmap as a public Github repo
Author: avourakis
Subreddit: datascience
Sentiment: positive
--------------------------------------------------
ID: 3
Title: SQL queries that group by number
Author: coronnial
Subreddit: datascience
Sentiment: negative
--------------------------------------------------
ID: 4
Title: Need help on analysis of AI performance, compute and time.
Author: PianistWinter8293
Subreddit: datascience
Sentiment: neutral
--------------------------------------------------
ID: 5
Title: I linked AI Performance Data with Compute Size Data and analyzed over Time
Author: PianistWinter8293
Subreddit: datascience
Sentiment: neutral
--------------------------------------------------
ID: 6
Title: Which position should I join? (Palan

## Task 6: After you have loaded data from a subreddit, choose a few more subreddit and load those!

Add cells if required

In [32]:
## Your code in this cell
## ------------------------
subreddits = ['NLP', 'datascienceproject', 'machinelearning', 'LanguageTechnology']

In [33]:
def get_vader_sentiment(text):
    sentiment_dict = analyzer.polarity_scores(text)
    return sentiment_dict['compound']

def preprocess_text(text):
    return text.lower()

In [34]:
for subreddit_name in subreddits:
    posts = []
    subreddit = reddit.subreddit(subreddit_name)
    
    for post in subreddit.hot(limit=100):
        posts.append({
            'id': post.id,
            'title': post.title,
            'author': str(post.author),
            'subreddit': post.subreddit.display_name,
            'url': post.url,
            'created': datetime.datetime.fromtimestamp(post.created),
            'processed_body': preprocess_text(post.selftext),
            'sentiment': get_vader_sentiment(post.selftext)
        })

In [35]:
insert_query = f"""
    INSERT INTO {schema}.data_science_posts (post_id, title, author, subreddit, title_link, timestamp, processed_body, sentiment)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (post_id) DO NOTHING;
    """
    
with connection, connection.cursor() as cursor:
    for post in posts:
        cursor.execute(insert_query, (
            post['id'],
            post['title'],
            post['author'],
            post['subreddit'],
            post['url'],
            post['created'],
            post['processed_body'],
            post['sentiment']
        ))

print(f"Data from subreddit '{subreddit_name}' loaded successfully.")

Data from subreddit 'LanguageTechnology' loaded successfully.



### In part II, we will search your database as `dsa_ro_user user`. To prepare your DB to be read, you will need to grant the dsa_ro_user schema access and select privileges on your table.

```SQL
GRANT USAGE ON SCHEMA <your schema> TO dsa_ro_user;  -- NOTE: change to your schema
GRANT SELECT ON <your table> TO dsa_ro_user;
```

In [36]:
GRANT_ACCESS=f"""
GRANT USAGE ON SCHEMA {schema} TO dsa_ro_user;
GRANT SELECT ON {schema}.data_science_posts TO dsa_ro_user;
"""

In [37]:
with connection, connection.cursor() as cursor:
    cursor.execute(GRANT_ACCESS)

In [38]:
# Commit and close the connection
connection.commit()
cur.close()
connection.close()

# Save your notebook, then `File > Close and Halt`

---